In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")

In [2]:
import matplotlib.pyplot as plt, argparse, numpy as np, math, sys, copy
from skimage.segmentation import slic, mark_boundaries, felzenszwalb
from skimage.util import img_as_float
from skimage import io
import numpy as np
import superpixels as sp
import plot as pl
import process_ground_truth as pgt
import glob
import eval_boundary as ev
import pipeline as pipe
import time
import superpixels as sp

In [3]:
ground_path = 'BSDS500/data/groundTruth/val/'
train_path = 'BSDS500/data/images/val/'
save_path = 'save/gslic/'
bound_prefix = save_path + 'images/'
eval_prefix = save_path + 'eval/'
extension = '.npz'

In [ ]:
imagePath = 'images/'
imageFile ='3063.jpg'
image = img_as_float(io.imread((imagePath + imageFile)))

seg_slic = slic(image, n_segments = 128, slic_zero = True)

s_image, n_segs, colors = sp.color_superpixel(image, seg_slic)

fig, (ax2, ax3) = plt.subplots(1,2, figsize=(20, 20))
ax2.imshow(image)
ax2.set_title("Original")
ax2.set_axis_off()
ax3.imshow(s_image)
ax3.set_title("Slic")
ax3.set_axis_off()
plt.show() 

In [ ]:
img = np.zeros(image.shape,dtype=np.uint8) #create blank image to save
img.fill(255)

cutz_images, cutz_nsegs = sp.generate_ultrametric_map(img, colors, seg_slic, n_segs
                                                      , step =3, start_at = 20, stop_at = 10)
print(len(cutz_images))

In [ ]:
ground_file ='3063.mat'
groundtruth = pgt.get_groundTruth(imagePath + ground_file)

ground_image = np.zeros(image.shape,dtype=np.uint8)
for i in range(0,3):
    ground_image[:, :, i] = groundtruth[:, :, 0]

img = np.zeros(image.shape,dtype=np.uint8) #create blank image to save
img.fill(255)
    
best_fmeasure = 0
for i in range(len(cutz_images)):
    precision, recall = ev.eval_bound(groundtruth, cutz_images[i], 0, True)
    fmeasure = 2 * ((precision * recall) / (precision + recall))
    
    if fmeasure > best_fmeasure:
        best_fmeasure = fmeasure
    
print(best_fmeasure)

### Read Groundtruth

In [4]:
len_ground = len(ground_path)
ground_files = glob.glob(ground_path + '*.mat')
ground_filenames = [x[len_ground:-4] for x in ground_files]

groundtruths, images, names = [], [], []

#load images
for filename in ground_filenames:
    ground_file = ground_path + filename + '.mat'
    train_file = train_path + filename + '.jpg'
    
    #read groundtruth and image
    groundtruth = pgt.get_groundTruth(ground_file)
    image = img_as_float(io.imread(train_file))
    
    #append image and groundtruths
    groundtruths.append(groundtruth)
    images.append(image)
    names.append(filename + '.png')
    
print(len(groundtruths), len(images))

100 100


### Generate boundaries and save results

In [5]:
ini = 0
fim = len(ground_filenames)

methods = ['hslic']

for method in methods:
    borders = []
    last_shape = None
    img = None
    index = 0
    
    starttime = time.time()
    for groundtruth, image, name in zip(groundtruths[ini:fim], images[ini:fim], names[ini:fim]):
        if(image.shape != last_shape):
            img = np.zeros(image.shape,dtype=np.uint8) #create blank image to save
            last_shape = image.shape
        
        seg_slic = slic(image, n_segments = 128, slic_zero = True)
        s_image, n_segs, colors = sp.color_superpixel(image, seg_slic)
        ultra_image = sp.generate_ultrametric_image(img, colors, seg_slic, n_segs
                                                    , step = 2, start_at = 15, stop_at = 1)
        
        #convert to grayscale
        gray_image = np.dot(ultra_image[...,:3], [0.299, 0.587, 0.114])
        
        #save image
        boundname = bound_prefix + name
        io.imsave(boundname, gray_image)
        
        #verbose mode
        index += 1
        if index % 10 == 0:
            print(index)

    endtime = time.time()
        
    #np.savez(boundname, borders=borders)
    print((endtime - starttime), 'seconds')

10
20
30
40
50
60
70
80
90
100
281.20791482925415 seconds


### Recover boundaries and evaluate results

In [6]:
ini = 0
fim = len(ground_filenames)

epsilon = 1e-4 #just to round down

methods = ['hslic']

for method in methods:    
    threshold = 0
    index = 0

    starttime = time.time()
    
    for groundtruth, name in zip(groundtruths[ini:fim], names[ini:fim]):
        precisions, recalls = [], []
        
        boundname = bound_prefix + name
        evalname = eval_prefix + name + extension
        
        image = img_as_float(io.imread(boundname))        
        unique = np.unique(image)
                
        for thresh in unique[1:]:
            thresh_image = image
            thresh_image[image < (thresh + epsilon)] = 0 #round down thresh
            thresh_image[image >= (thresh + epsilon)] = 1

            #calculate values from precision and recall
            precision, recall = ev.eval_bound(groundtruth[:,:,0], thresh_image, threshold, True)
            precisions.append(precision)
            recalls.append(recall)
        
        #verbose mode
        index += 1
        if index % 10 == 0:
            print(index)
        
        #save precision file
        np.savez(evalname, precisions=precisions, recalls=recalls)
            
    endtime = time.time()
    print((endtime - starttime), 'seconds')

10
20
30
40
50
60
70
80
90
100
1320.4655997753143 seconds


### Load evaluations and calculate f-measures

In [7]:
methods = ['hslic']
fmeasures = []

for method in methods:
    precisions, recalls = [], []
    
    eval_files = glob.glob(eval_prefix + '*' + extension)
    
    for filename in eval_files:
        npload = np.load(filename)
        precisions.append(npload['precisions'])
        recalls.append(npload['recalls'])

    len_precisions = 1000000
        
    for i in range(len(precisions)):
        if (len(precisions[i]) < len_precisions):
            len_precisions = len(precisions[i])
            
    for i in range(0,len_precisions):
        cutz_precision, cutz_recall = [], []
        
        for precision, recall in zip(precisions, recalls):
            cutz_precision.append(precision[i])
            cutz_recall.append(recall[i])
    
        np_precisions = (np.array(cutz_precision))
        np_recalls = (np.array(cutz_recall))

        fmeasure = 2 * ((np_precisions * np_recalls) / (np_precisions + np_recalls))
        fmeasures.append(fmeasure)
        
print(np.average(fmeasures, axis=1))

[ 0.10947253  0.10947253  0.10947253  0.10947253  0.10947253  0.10947253]
